In [ ]:
!pip install tensorflow==2.3.0

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

In [2]:
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [14]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images = ((train_images / 255.0) - 0.1307)/0.3081
test_images = ((test_images / 255.0) - 0.1307)/0.3081
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [17]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

opt = tf.optimizers.SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
	datagen = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
	it_train = datagen.flow(train_images, train_labels, batch_size=64)
	steps = int(train_images.shape[0] / 64)
	history = model.fit(it_train, steps_per_epoch=steps, epochs=10, validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

In [35]:
model.save("network.h5")

In [ ]:
model.save("network")
converter = tf.lite.TFLiteConverter.from_saved_model("/content/network")
model_no_quant_tflite = converter.convert()


open("network.tflite", "wb").write(model_no_quant_tflite)

In [37]:
def representative_dataset():
  for i in range(500):
    yield([test_images[i].reshape(1, 32, 32, 3).astype(np.float32)])

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

open("network_with_quant.tflite", "wb").write(model_tflite)

301104